In [ ]:
#@title <center><img src="https://i.imgur.com/Vz0uo7C.png" width="275">
OMDB_API         = "c42fc7df" #@param {type:"string"} 
REMOTE_NAME      = ""         #@param {type:"string"} 
SHARED_DRIVE     = ""         #@param {type:"string"}
RCLONE_PASSWORD  = ""         #@param {type:"string"}
HIDE_POST_COUNT  = 25         #@param {type:"integer"} 
HIDE_REACT_SCORE = 25         #@param {type:"integer"} 
 
from google.colab import files
from IPython.display import clear_output
 
## Install FFmpeg, MediaInfo, rClone
!wget -q "https://github.com/InspectorWilliamHenderson/rFFmedia/releases/latest/download/rFFmedia.zip" && unzip -o -qq "./rFFmedia.zip" -d "/" && rm -rf "/content/sample_data" "./rFFmedia.zip"
 
## Configure rClone
!echo "Upload the rClone.conf"
uploaded = files.upload()
clear_output()
!rclone --config '' move "./rclone.conf" "/root/.config/rclone/"
 
## rClone Mount
if SHARED_DRIVE != "": SHARED_DRIVE = "--drive-root-folder-id=" + SHARED_DRIVE
if RCLONE_PASSWORD != "":
  %env RCLONE_CONFIG_PASS=$RCLONE_PASSWORD
  clear_output()
!mkdir "/content/{REMOTE_NAME}/"
!rclone mount "{REMOTE_NAME}": "/content/{REMOTE_NAME}/" --buffer-size 32M --daemon $SHARED_DRIVE --max-read-ahead 32M --poll-interval 30s --read-only --use-mmap --vfs-cache-mode full --vfs-read-chunk-size-limit 16M --vfs-read-chunk-size 8M -v

In [ ]:
#@title <h3><b><center>Movie and TV Shows Template
import base64, os, regex, requests, subprocess, threading
 
def CreateScreenshots(PATH, TIME):
  global SCREENSHOTS
  SCREENSHOT_LINK = subprocess.Popen(['ffmpeg', '-hide_banner', '-loglevel', 'error', '-ss', TIME, '-i', PATH, '-vframes', '1', '-q:v', '2', '-y', f'./frame_{TIME}.jpeg'], stdin=open(os.devnull, 'wb'), stdout=open(os.devnull, 'wb')).communicate()
  if os.path.exists(f'./frame_{TIME}.jpeg'):
    SCREENSHOT_LINK = requests.post("https://api.imgbb.com/1/upload",{"key":"6b1c29d7993f76b154e2c704f1d13879","image":base64.b64encode(open(f"./frame_{TIME}.jpeg", "rb").read())})
    if str(SCREENSHOT_LINK.status_code) == "200": SCREENSHOTS += "[URL='{}'][IMG WIDTH='800px']{}[/IMG][/URL]\n".format(SCREENSHOT_LINK.json()['data']['url'], SCREENSHOT_LINK.json()['data']['url'])
 
def InitiateScreenshots():
  global SCREENSHOTS
  if SCREENSHOTS == True:
    if (".DoVi." in os.path.basename(PATH)) or (".DV." in os.path.basename(PATH)): SCREENSHOTS = ""; return None
    SCREENSHOTS = "[HR][/HR][INDENT][SIZE=6][FORUMCOLOR][B]Media Screenshots :[/B][/FORUMCOLOR][/SIZE][/INDENT]\n[SPOILER='Media Screenshots'][CENTER]"
    CreateScreenshots(PATH, "02:30.00")
    CreateScreenshots(PATH, "05:00.00")
    CreateScreenshots(PATH, "07:30.00")
    SCREENSHOTS += "[/CENTER][/SPOILER]"
    if SCREENSHOTS == "[HR][/HR][INDENT][SIZE=6][FORUMCOLOR][B]Media Screenshots :[/B][/FORUMCOLOR][/SIZE][/INDENT]\n[SPOILER='Media Screenshots'][CENTER][/CENTER][/SPOILER]": SCREENSHOTS = ""
  else: SCREENSHOTS = ""
 
def GenerateLink(PATH, FOLDER):
  global LINK
  LINK = PATH.replace('/content/'+REMOTE_NAME+'/', '')
  if FOLDER == "PARENT FOLDER": LINK = os.path.dirname(LINK)
  if FOLDER == "GRANDPARENT FOLDER": LINK = os.path.dirname(os.path.dirname(LINK))
  if SHARED_DRIVE == "": LINK = subprocess.check_output(['rclone', 'link', f'{REMOTE_NAME}:{LINK}']).decode().strip()
  else: LINK = subprocess.check_output(['rclone', 'link', f'{REMOTE_NAME}:{LINK}', SHARED_DRIVE]).decode().strip()
  LINK = "[HIDEREACT=1,2,3,4,5,6,7,8][DOWNCLOUD]" + ''.join(LINK) + "[/DOWNCLOUD][/HIDEREACT]"
  if HIDE_REACT_SCORE != 0: LINK = "[HIDEREACTSCORE={}]{}[/HIDEREACTSCORE]".format(HIDE_REACT_SCORE, LINK)
  if HIDE_POST_COUNT != 0: LINK = "[HIDEPOSTS={}]{}[/HIDEPOSTS]".format(HIDE_POST_COUNT, LINK)
 
def GetIMDb(PATH):
  global IMDB
  if IMDB == "":
    try: YEAR = "&y="+regex.findall(r"(19\d{2}|20\d{2})", PATH)[-1]
    except: YEAR = ""
    TITLE = regex.sub('[^a-zA-Z0-9]', '+', os.path.basename(PATH)).split(f"+{YEAR.replace('&y=','')}+")[0].split("+S0")[0].split("+S1")[0].split("+S2")[0].split("+UHD")[0].split("+1080p")[0].split("+2160p")[0].split("+HYBRID")[0].split("+EXTENDED")[0]
    IMDB = requests.get(f'http://www.omdbapi.com/?t={TITLE}{YEAR}&apikey={OMDB_API}&r=json').json()
  else: IMDB = requests.get(f'http://www.omdbapi.com/?i=tt{regex.search(r"tt([0-9]+)", IMDB)[1].lstrip("0").zfill(7)}&apikey={OMDB_API}&r=json').json()
 
def GetMediaInfo(PATH):
  global MEDIAINFO
  COMMUNICATE = subprocess.Popen(['mediainfo', '--logfile=./.nfo', PATH], stdin=open(os.devnull, 'wb'), stdout=open(os.devnull, 'wb')).communicate()
  MEDIAINFO = open('./.nfo', 'r', encoding='utf-8').readlines(); MEDIAINFO.pop(); del MEDIAINFO[1]; MEDIAINFO[1] = "Complete Name                            : {}\n".format(os.path.basename(PATH)); MEDIAINFO = ''.join(MEDIAINFO)
 
def BlackPearl_Posting_Template():
  global FOLDER, IMDB, PATH,  SCREENSHOTS, YOUTUBE
  subprocess.run(['rm', '-f', './.nfo', './frame*.jpeg'])
  InitSS  = threading.Thread(target=InitiateScreenshots())
  GenLink = threading.Thread(target=GenerateLink(PATH, FOLDER))
  GetIMDB = threading.Thread(target=GetIMDb(PATH))
  GetMI   = threading.Thread(target=GetMediaInfo(PATH))
  GetIMDB.start()
  GetMI.start()
  InitSS.start()
  GenLink.start()
  YOUTUBE = "" if YOUTUBE == "" else regex.findall(r"([A-Za-z0-9-_]{11})", YOUTUBE)[-1]
  GetIMDB.join()
  try:
    if IMDB['Response'] == 'False':
      print("Give IMDb ID")
      return None
  except: pass
  BBCODE  = "[CENTER][URL='{}'][IMG WIDTH='350px']{}[/IMG][/URL][/CENTER]\n".format(regex.sub("_V1_SX\d+.jpg","_V1_SX2000.png", IMDB['Poster']), regex.sub("_V1_SX\d+.jpg","_V1_SX2000.png", IMDB['Poster'])) if IMDB['Poster'] != "N/A" else ""
  BBCODE += "[CENTER][URL='https://blackpearl.biz/search/1/?q={}&o=date'][FORUMCOLOR][B][SIZE=6]{} ({})[/SIZE][/B][/FORUMCOLOR][/URL][/CENTER]\n".format(IMDB['imdbID'], IMDB['Title'], IMDB['Year'])
  BBCODE += "[CENTER][URL='https://imdb.com/title/{}'][IMG WIDTH='46px']https://ia.media-imdb.com/images/M/MV5BMTk3ODA4Mjc0NF5BMl5BcG5nXkFtZTgwNDc1MzQ2OTE@.png[/IMG][/URL][/CENTER]".format(IMDB['imdbID'])
  BBCODE += "[HR][/HR][INDENT][SIZE=6][FORUMCOLOR][B]Plot Summary :[/B][/FORUMCOLOR][/SIZE]\n\n[JUSTIFY]{}[/JUSTIFY][/INDENT]".format(IMDB['Plot']) if IMDB['Plot'] != "N/A" else ""
  BBCODE += "[HR][/HR][INDENT][SIZE=6][FORUMCOLOR][B]IMDb Info :[/B][/FORUMCOLOR][/SIZE][/INDENT]\n[LIST]"
  BBCODE += "[*][FORUMCOLOR][B]Rating :[/B][/FORUMCOLOR] {}\n".format(IMDB['Rated']) if IMDB['Rated'] != "N/A" else ""
  BBCODE += "[*][FORUMCOLOR][B]Genre :[/B][/FORUMCOLOR] {}\n".format(IMDB['Genre']) if IMDB['Genre'] != "N/A" else ""
  BBCODE += "[*][FORUMCOLOR][B]Directed By :[/B][/FORUMCOLOR] {}\n".format(IMDB['Director']) if IMDB['Director'] != "N/A" else ""
  BBCODE += "[*][FORUMCOLOR][B]Written By :[/B][/FORUMCOLOR] {}\n".format(IMDB['Writer']) if IMDB['Writer'] != "N/A" else ""
  BBCODE += "[*][FORUMCOLOR][B]Starring :[/B][/FORUMCOLOR] {}\n".format(IMDB['Actors']) if IMDB['Actors'] != "N/A" else ""
  BBCODE += "[*][FORUMCOLOR][B]Release Date :[/B][/FORUMCOLOR] {}\n".format(IMDB['Released']) if IMDB['Released'] != "N/A" else ""
  BBCODE += "[*][FORUMCOLOR][B]Awards :[/B][/FORUMCOLOR] {}[/LIST]".format(IMDB['Awards']) if IMDB['Awards'] != "N/A" else "[/LIST]"
  BBCODE += "[HR][/HR][INDENT][SIZE=6][FORUMCOLOR][B]Official Trailer :[/B][/FORUMCOLOR][/SIZE][/INDENT]\n\n[CENTER][MEDIA=YouTube]{}[/MEDIA][/CENTER]".format(YOUTUBE) if YOUTUBE != "" else ""
  BBCODE += "[HR][/HR][INDENT][SIZE=6][FORUMCOLOR][B]Media Info :[/B][/FORUMCOLOR][/SIZE][/INDENT]"
  GetMI.join()
  InitSS.join()
  BBCODE += "[SPOILER='Media Info'][CODE TITLE='Media Info']{}[/CODE][/SPOILER]\n{}".format(MEDIAINFO, SCREENSHOTS)
  GenLink.join()
  BBCODE += "[HR][/HR][INDENT][SIZE=6][FORUMCOLOR][B]Download Link :[/B][/FORUMCOLOR][/SIZE][/INDENT]\n[CENTER]{}[/CENTER]".format(LINK)
  print(BBCODE)
  subprocess.run(['rm', '-f', './.nfo', './frame*.jpeg'])

In [ ]:
IMDB        = ""         #@param {type:"string"}
YOUTUBE     = ""         #@param {type:"string"}
PATH        = ""         #@param {type:"string"}
FOLDER      = "Disabled" #@param ["Disabled", "PARENT FOLDER", "GRANDPARENT FOLDER"]
SCREENSHOTS = False      #@param {type:"boolean"}
 
BlackPearl_Posting_Template()